In [1]:
import pdb
import torch
import cv2
import os
from PIL import Image
from skimage import io, transform
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from torchvision.datasets.folder import pil_loader
import random
import numpy as np
from torchsummary import summary
import pretrainedmodels
from sklearn.model_selection import KFold, StratifiedKFold


from albumentations import torch as AT
os.environ['CUDA_VISIBLE_DEVICES']=""
%matplotlib inline

In [2]:
!ls data

sample_submission.csv  test  train  train_labels.csv


In [3]:
train_df = pd.read_csv('data/train_labels.csv')
test_df = pd.read_csv('data/sample_submission.csv')

In [4]:
test_df.shape

(57458, 2)

In [ ]:
train_df.head()

In [ ]:
train_df.at[0, 'label'] = 1

In [ ]:
kfold = StratifiedKFold(20, shuffle=True, random_state=69)

In [ ]:
kfold.split(train_df['id'], train_df['label'])

In [ ]:
train_df.iloc[test]

In [ ]:
train_df.head()

In [ ]:
train_df.info(), test_df.info()

In [ ]:
%%time
img = mpimg.imread('data/train/00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif')

In [ ]:
timeit mpimg.imread('data/train/00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif')

In [ ]:
timeit pil_loader('data/train/00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif')

In [ ]:
timeit cv2.imread('data/train/00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif', flags=cv2.IMREAD_COLOR)

In [ ]:
timeit io.imread("data/train/00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif")

##### use cv2.imread for reading images 

In [ ]:
img.shape


In [ ]:
plt.imshow(img)

In [ ]:
# plot some random training images with their ground truth labels
i = random.choice(range(train_df.shape[0]))
img = mpimg.imread('data/train/' + train_df.iloc[i]['id'] + '.tif')
plt.imshow(img)
plt.title(train_df.iloc[i]['label'])
plt.show()

In [ ]:
train_df['label'].value_counts()

In [ ]:
train_df['label'].hist

In [ ]:
img = mpimg.imread('data/train/dd6dfed324f9fcb6f93f46f32fc800f2ec196be2.tif')
plt.imshow(img)
plt.show();
aug_img = train_aug(image=img)
plt.imshow(aug_img['image'])
plt.show();

#### Dataloader analysis


In [ ]:
from dataloader import get_dataloader

In [ ]:
def unnormalize(img):
    mean = torch.as_tensor([0.485, 0.456, 0.406])
    std = torch.as_tensor([0.229, 0.224, 0.225])
    #return (img * std) + mean
    for t, m, s in zip(img, mean, std):    
        t.mul_(s).add_(m)
    return img

In [ ]:
phase = "train"
dataloader = get_dataloader(phase)

In [ ]:
batch = next(iter(dataloader))

In [ ]:
images, labels = batch
labels

In [ ]:
# plot some random training images with their ground truth labels
i = random.choice(range(len(images)))
img = images[i].clone()
img = unnormalize(img)
plt.imshow(img.permute(1, 2, 0))
plt.title(labels[i].item())
plt.show()

In [ ]:
mean = torch.as_tensor([0.485, 0.456, 0.406])
std = torch.as_tensor([0.229, 0.224, 0.225])
#return (img * std) + mean
#return img.mul_(std).add_(mean)
labels[i].item()

In [ ]:
mean, std, mean.shape, std.shape

In [6]:
import pretrainedmodels
pretrainedmodels.model_names

['fbresnet152',
 'bninception',
 'resnext101_32x4d',
 'resnext101_64x4d',
 'inceptionv4',
 'inceptionresnetv2',
 'alexnet',
 'densenet121',
 'densenet169',
 'densenet201',
 'densenet161',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnet101',
 'resnet152',
 'inceptionv3',
 'squeezenet1_0',
 'squeezenet1_1',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vgg16_bn',
 'vgg19_bn',
 'vgg19',
 'nasnetamobile',
 'nasnetalarge',
 'dpn68',
 'dpn68b',
 'dpn92',
 'dpn98',
 'dpn131',
 'dpn107',
 'xception',
 'senet154',
 'se_resnet50',
 'se_resnet101',
 'se_resnet152',
 'se_resnext50_32x4d',
 'se_resnext101_32x4d',
 'cafferesnet101',
 'pnasnet5large',
 'polynet']

In [7]:
model_name='se_resnext50_32x4d'
model = pretrainedmodels.__dict__[model_name](num_classes=1000)

In [ ]:
model

In [ ]:
model.layer0.parameters()
model.layer1.parameters()
model.layer2.parameters()
model.layer3.parameters()
model.layer4.parameters()
#list(model.children())[0]#.parameterss()
model.avg_pool

In [ ]:
[
    {"params": model.layer0.paramters()},
    {"params": model.layer1.paramters()},
    {"params": model.layer2.paramters()},
    {"params": model.layer3.paramters()},
    {"params": model.layer4.paramters()},
    {"params": model.avg_pool.paramters()},
    {"params": model.last_linear.parameters(), "lr": 3e-5}
], lr=3e-5*0.001,

import torch

In [13]:
blocks = list(model.children())
head = torch.nn.Sequential(*list(blocks[0].children())[:-1])
model._modules['layer0'] = head
#model._modules['avg_pool'] = torch.nn.AvgPool2d(kernel_size=5, stride=1, padding=0) 
#model._modules['avg_pool'] = torch.nn.Sequential(*[torch.nn.ConstantPad2d((0,1,0,1), 0), torch.nn.AvgPool2d(kernel_size=7, stride=1, padding=0)])
model._modules['avg_pool'] = torch.nn.Sequential(torch.nn.AdaptiveAvgPool2d((1, 1)))
model._modules['last_linear'] = torch.nn.Linear(in_features=2048, out_features=1, bias=True)

In [12]:
torch.nn.AdaptiveAvgPool2d?

In [44]:
#head.parameters()
#for block in blocks:
base = list(blocks[0].children())[:-1] + blocks[1:-2]
torch.nn.Sequential(*base)
torch.nn.Dropout?
#    print(block)
#    print('*'*100)

In [46]:

torch.nn.Linear?

In [ ]:
#blocks[0] = head

In [ ]:
#blocks[-1] = torch.nn.Linear(in_features=2048, out_features=1, bias=True)

In [ ]:
#model = torch.nn.Sequential(*blocks)

In [ ]:
#model._modules

In [14]:
#img = torch.Tensor(1, 3, 224, 224)#.to('cuda')
img = torch.Tensor(1, 3, 112, 112)
img = torch.Tensor(1, 3, 96, 96)

In [15]:
out = model(img)

print(out)

tensor([[-0.2230]], grad_fn=<AddmmBackward>)


In [16]:
x = None
for param in model.children():
    
    print(param)
    if x is None:
        x = param(img)
    else:
        x = param(x)
    print(x.shape)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
)
torch.Size([1, 64, 48, 48])
Sequential(
  (0): SEResNeXtBottleneck(
    (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (se_module): SEModule(
      (avg_pool): AdaptiveAvgPool2d(output_size=1)
      (fc1): Conv2d(256, 16, kernel_size=(1, 1), stride=(1, 1))
      (relu): ReLU(inplace)
      (fc2): Conv2d(16, 256, kernel_size=(1, 1), stride=(1, 1))
      (sigmoid): Sigmoid()
    )
    (dow

RuntimeError: size mismatch, m1: [2048 x 1], m2: [2048 x 1] at /opt/conda/conda-bld/pytorch_1549636813070/work/aten/src/TH/generic/THTensorMath.cpp:940

In [17]:
summary(model, input_size=(3, 96, 96))
#summary(model, input_size=(3, 112, 112))
#summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 48, 48]           9,408
            Conv2d-2          [-1, 128, 48, 48]           8,192
       BatchNorm2d-3          [-1, 128, 48, 48]             256
              ReLU-4          [-1, 128, 48, 48]               0
            Conv2d-5          [-1, 128, 48, 48]           4,608
       BatchNorm2d-6          [-1, 128, 48, 48]             256
              ReLU-7          [-1, 128, 48, 48]               0
            Conv2d-8          [-1, 256, 48, 48]          32,768
       BatchNorm2d-9          [-1, 256, 48, 48]             512
           Conv2d-10          [-1, 256, 48, 48]          16,384
      BatchNorm2d-11          [-1, 256, 48, 48]             512
AdaptiveAvgPool2d-12            [-1, 256, 1, 1]               0
           Conv2d-13             [-1, 16, 1, 1]           4,112
             ReLU-14             [-1, 1

In [ ]:
 
from albumentations import *

In [ ]:
train_aug = Compose([
    Resize(224, 224),
    RandomRotate90(p=0.5),
    Transpose(p=0.5),
    Flip(p=0.5),
    OneOf([CLAHE(clip_limit=2), IAASharpen(), IAAEmboss(), RandomBrightnessContrast(), JpegCompression(), Blur(), GaussNoise() ], p=0.5), 
    HueSaturationValue(p=0.5), 
    ShiftScaleRotate(shift_limit=0.15, scale_limit=0.15, rotate_limit=45, p=0.5),
    Normalize(p=1)
])
val_aug = Compose([
    Normalize(p=1)
])


In [ ]:
ShiftScaleRotate?

In [ ]:
Normalize?

In [ ]:
train_aug?

##### AUC ROC

In [ ]:
# roc curve and auc
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, weights=[1,1], random_state=1)
# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)
# fit a model
model = KNeighborsClassifier(n_neighbors=3)
model.fit(trainX, trainy)
# predict probabilities
probs = model.predict_proba(testX)
# keep probabilities for the positive outcome only
probs = probs[:, 1]


In [ ]:
testy.shape, probs.shape

In [ ]:
fpr, tpr, thresholds

In [ ]:
# calculate AUC
auc = roc_auc_score(testy, probs)
print('AUC: %.3f' % auc)
# calculate roc curve
fpr, tpr, thresholds = roc_curve(testy, probs)
# plot no skill
pyplot.plot([0, 1], [0, 1], linestyle='--')
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
pyplot.plot(fpr, tpr, marker='.')
# show the plot
pyplot.show()

In [ ]:
pyplot.savefig('lol.png')

In [ ]:
pyplot.close('all')

In [47]:
import apex

In [48]:
apex.amp.?